In [4]:
#Import libraries

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense


In [6]:
# Path to the model weights files

weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'

# Image dimension 
img_width, img_height = 150, 150

# Path to directories 
train_data_dir = 'Downloads/Data/train'
validation_data_dir = 'Downloads/Data/validation'  
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 5
batch_size = 16


In [7]:
# Build the VGG16 network

basemodel = applications.VGG16(weights='imagenet', include_top=False, input_shape= (150,150,3)) 
print('Model loaded.')

# Build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=basemodel.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

'''it is necessary to start with a fully-trained
classifier, including the top classifier, in order to successfully do fine-tuning'''

top_model.load_weights(top_model_weights_path)

# Add the model on top of the convolutional base
model = Model(input = basemodel.input, output= top_model(basemodel.output))
 
# Set the first 25 layers (up to the last conv block) to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# Compile the model with a SGD/momentum optimizer and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# Prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary') 

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)


Model loaded.
Found 2000 images belonging to 2 classes.


/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


Found 800 images belonging to 2 classes.
Epoch 1/5


/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=5, validation_data=<keras_pre..., steps_per_epoch=125, validation_steps=800)`


125/125 [==============================] - 1537s 12s/step - loss: 0.3631 - acc: 0.9220 - val_loss: 0.6153 - val_acc: 0.8975
Epoch 2/5
125/125 [==============================] - 1525s 12s/step - loss: 0.3495 - acc: 0.9220 - val_loss: 0.6153 - val_acc: 0.8975
Epoch 3/5
125/125 [==============================] - 1463s 12s/step - loss: 0.3184 - acc: 0.9275 - val_loss: 0.6153 - val_acc: 0.8975
Epoch 4/5
125/125 [==============================] - 1461s 12s/step - loss: 0.3600 - acc: 0.9215 - val_loss: 0.6153 - val_acc: 0.8975
Epoch 5/5
125/125 [==============================] - 1531s 12s/step - loss: 0.3638 - acc: 0.9150 - val_loss: 0.6153 - val_acc: 0.8975
